In [320]:
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf

import tensorflow_probability as tfp
tfd = tfp.distributions

from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, BatchNormalization
from tensorflow.keras.layers import ReLU, Dropout
from tensorflow.keras.optimizers import SGD
from silence_tensorflow import silence_tensorflow

silence_tensorflow()

from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error


In [321]:
def calc_rmse(y_pred,y_true):
    y_pred_flat = np.asarray(y_pred).flatten()
    y_true_flat = np.asarray(y_true).flatten()
    rmse = np.sqrt(mean_squared_error(y_true_flat, y_pred_flat))
    return rmse

In [322]:
#Initialization of neural network according to Xavier initialization
def init(shape):
    return tf.random.truncated_normal(
        shape, 
        mean=0.0,
        stddev=np.sqrt(2/sum(shape)))

In [323]:
class BayesianDenseLayer(tf.keras.Model):
    def __init__(self, input_data, output_data, name=None):
        
        super(BayesianDenseLayer, self).__init__(name=name)
        self.input_data = input_data
        self.output_data = output_data
        
        self.weight_loc = tf.Variable(init([input_data, output_data]), name='weight_loc')
        self.weight_std = tf.Variable(init([input_data, output_data]) -6.0, name='weight_std')
        self.bias_loc = tf.Variable(init([1, output_data]), name='bias_loc')
        self.bias_std = tf.Variable(init([1, output_data])-6.0, name='bias_std')
    
    
    def call(self, x, sampling=True):
        """Perform the forward pass"""
        
        if sampling:
        
            # Flipout-estimated weight samples
            s = tfp.random.rademacher(tf.shape(x))
            r = tfp.random.rademacher([x.shape[0], self.output_data])
            w_samples = tf.nn.softplus(self.weight_std)*tf.random.normal([self.input_data, self.output_data])
            w_perturbations = r*tf.matmul(x*s, w_samples)
            w_outputs = tf.matmul(x, self.weight_loc) + w_perturbations
            
            # Flipout-estimated bias samples
            r = tfp.random.rademacher([x.shape[0], self.output_data])
            b_samples = tf.nn.softplus(self.bias_std)*tf.random.normal([self.output_data])
            b_outputs = self.bias_loc + r*b_samples
            
            return w_outputs + b_outputs
        
        else:
            return x @ self.weight_loc + self.bias_loc
    
    @property
    def losses(self):
        """Sum of the KL divergences between priors + posteriors"""
        weight = tfd.Normal(self.weight_loc, tf.nn.softplus(self.weight_std))
        bias = tfd.Normal(self.bias_loc, tf.nn.softplus(self.bias_std))
        prior = tfd.Normal(0, 1)
        return (tf.reduce_sum(tfd.kl_divergence(weight, prior)) +
                tf.reduce_sum(tfd.kl_divergence(bias, prior)))

In [324]:
class BayesianDenseNetwork(tf.keras.Model):
    
    def __init__(self, dims, name=None):
        
        super(BayesianDenseNetwork, self).__init__(name=name)
        
        self.steps = []
        self.acts = []
        for i in range(len(dims)-1):
            self.steps += [BayesianDenseLayer(dims[i], dims[i+1])]
            self.acts += [tf.nn.relu]
            
        self.acts[-1] = lambda x: x
        
    
    def call(self, x, sampling=True):

        for i in range(len(self.steps)):
            x = self.steps[i](x, sampling=sampling)
            x = self.acts[i](x)
            
        return x
    
    @property
    def losses(self):
        """Sum of the KL divergences between priors + posteriors"""
        return tf.reduce_sum([s.losses for s in self.steps])

In [325]:
class BNN_Reg(tf.keras.Model):
    
    def __init__(self, dims, name=None):
        
        super(BNN_Reg, self).__init__(name=name)
        
        # Multilayer fully-connected neural network to predict mean
        self.loc_mean = BayesianDenseNetwork(dims)
        
        # Variational distribution variables for observation error
        self.std_alpha = tf.Variable([10.0], name='std_alpha')
        self.std_beta = tf.Variable([10.0], name='std_beta')

    
    def call(self, x, sampling=True):
        
        # Predict means
        loc_preds = self.loc_mean(x, sampling=sampling)
    
        # Predict std deviation
        post_dist = tfd.Gamma(self.std_alpha, self.std_beta)
        adjust = lambda x: tf.sqrt(tf.math.reciprocal(x))
        N = x.shape[0]
        if sampling:
            std_preds = adjust(post_dist.sample([N]))
        else:
            std_preds = tf.ones([N, 1])*adjust(post_dist.mean())
    
        # Return mean and std predictions
        return tf.concat([loc_preds, std_preds], 1)
    
    
    def ll(self, x, y, sampling=True):
        mean_std = self.call(x, sampling=sampling)
        return tfd.Normal(mean_std[:,0], mean_std[:,1]).log_prob(y[:,0])
    
    def Normal_Sampling(self, x):
        preds = self.call(x)
        return tfd.Normal(preds[:,0], preds[:,1]).sample()
    
    def sampling(self, x, n = 1):
        sampling = np.zeros((x.shape[0], n))
        for k in range(n_samples):
            sampling[:,k] = self.Normal_Sampling(x)
        return sampling
    
    @property
    def loss(self):
        
        mean_loss = self.loc_mean.losses
        post_dist = tfd.Gamma(self.std_alpha, self.std_beta)
        prior = tfd.Gamma(10.0, 10.0)
        std_loss = tfd.kl_divergence(post_dist, prior)

        # Return the sum of both
        return mean_loss + std_loss

In [326]:
def process(model, optimizer, x_data, y_data, N):  
    #calculating lower bound (elbo method)
    with tf.GradientTape() as gradtape:
        ll = model.ll(x_data, y_data)
        model_loss = model.loss 
        train_cost = model_loss/N - tf.reduce_mean(ll)
    derivatives = gradtape.gradient(train_cost, model.trainable_variables)
    optimizer.apply_gradients(zip(derivatives, model.trainable_variables))
    return train_cost

In [327]:
def perform(model, optimizer, cycles, train_data, test_data, N):
    train_elbo = np.zeros(cycles)
    mse_root = []
    y_pred = []; y = []
    for ep in range(cycles):
            #Update weights each batch
        for X_values, y_values in train_data:
            train_elbo[ep] += process(model, optimizer, X_values, y_values, N)  

    # Evaluate performance on validation data
        for X_values, y_values in test_data:
            y_pred.append(model(X_values, sampling=False)[:, 0])
            y.append(y_values)
        mse_root.append(calc_rmse(np.asarray(y_pred), np.asarray(y)))
    return mse_root[-1]

In [328]:
weather_ds = pd.read_csv("seattle-weather.csv")


In [329]:
weather_ds.describe()

,precipitation,temp_max,temp_min,wind
count,1461.000000,1461.000000,1461.000000,1461.000000
mean,3.029432,16.439083,8.234771,3.241136
std,6.680194,7.349758,5.023004,1.437825
min,0.000000,-1.600000,-7.100000,0.400000
25%,0.000000,10.600000,4.400000,2.200000
50%,0.000000,15.600000,8.300000,3.000000
75%,2.800000,22.200000,12.200000,4.000000
max,55.900000,35.600000,18.300000,9.500000


In [330]:
weather_ds = weather_ds
weather_ds_copy = weather_ds
data_dummies = weather_ds_copy.drop(columns=['date', 'precipitation', 'temp_max', 'temp_min', 'wind' ])
data_dummies = pd.get_dummies(data=data_dummies).astype(int)
weather_ds_copy = weather_ds_copy[['precipitation', 'wind', 'temp_min', 'temp_max']]
weather_ds = pd.concat([data_dummies,weather_ds_copy], axis = 1)

In [331]:
weather_ds.head()

,weather_drizzle,weather_fog,weather_rain,weather_snow,weather_sun,precipitation,wind,temp_min,temp_max
0,1,0,0,0,0,0.0,4.7,5.0,12.8
1,0,0,1,0,0,10.9,4.5,2.8,10.6
2,0,0,1,0,0,0.8,2.3,7.2,11.7
3,0,0,1,0,0,20.3,4.7,5.6,12.2
4,0,0,1,0,0,1.3,6.1,2.8,8.9


In [332]:
X = np.asarray(weather_ds[['weather_drizzle','weather_fog', 'weather_rain', 'weather_snow', 'weather_sun','precipitation', 'wind', 'temp_min']])
y = np.asarray(weather_ds['temp_max'])
X = X.astype('float32')
y = y.astype('float32')

In [333]:
def K_Fold_Cross_Validation(cycles, learning_rate, l1_neurons, n_folds = 5):
    performance = []          
   
    kf = KFold(n_splits =n_folds, shuffle = True, random_state = 42)
    test_index_set = 0
    for train_index, test_index in kf.split(X):
        test_index_set += 1
        x_train, x_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]
          
        N_train = x_train.shape[0]
        N_val = x_test.shape[0]
        # Make y 2d
        y_train = np.expand_dims(y_train, 1)
        y_test = np.expand_dims(y_test, 1)
        data_train = tf.data.Dataset.from_tensors((x_train, y_train))
        data_test = tf.data.Dataset.from_tensors((x_test, y_test))
        BNN_model = BNN_Reg([np.shape(X)[1], l1_neurons, 1])
        optimizer = tf.keras.optimizers.SGD(learning_rate= learning_rate) #gradient descent
        root_mse = perform(BNN_model, optimizer, cycles, data_train, data_test, x_train.shape[0])
        performance.append(root_mse)
        print(f"Fold: {test_index_set}")
        print(f"Root MSE: {root_mse}")

    print("Mean Performance:", np.mean(performance))

    return np.mean(performance)


In [334]:
#Number of nodes in the hidden layer:
hl_nodes = [int(x) for x in np.linspace(10,100, num = 10)]
#Number of learning cycles:
epochs = [int(x) for x in np.linspace(50,1000, num = 5)]
#learning_rate
learning_rate = [float(x) for x in np.linspace(0.01,0.0001, num = 3)] 
random_grid = {'hl_nodes': hl_nodes,
               'epochs' : epochs,
               'learning_rate': learning_rate
               }
print(random_grid)

{'hl_nodes': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100], 'epochs': [50, 287, 525, 762, 1000], 'learning_rate': [0.01, 0.00505, 0.0001]}


In [335]:
def random_search_BNN(hl_nodes,epochs,learning_rate):
    info = []
    for nodes in hl_nodes:
        for eps in epochs:
            for lr in learning_rate:
                rmse = K_Fold_Cross_Validation(eps, lr, nodes)
                info.append(rmse)
                print("Processed %0.3f out of %0.3f elements" % (len(info),len(epochs)*len(hl_nodes)*len(learning_rate)))

    opt = np.min(info,axis = 0)
    return print("Number of nodes: %0.3f, number of learning cycles: %0.3f, learning rate: %0.3f" %(opt[1], opt[2], opt[3]))
  

In [336]:
random_search_BNN(hl_nodes,epochs,learning_rate)

Fold: 1
Root MSE: 9.59655475616455
Fold: 2
Root MSE: 7.054793834686279
Fold: 3
Root MSE: 8.04556941986084
Fold: 4
Root MSE: 9.502260208129883
Fold: 5
Root MSE: 10.311676979064941
Mean Performance: 8.902171
Processed 1.000 out of 150.000 elements
Fold: 1
Root MSE: 4.697515487670898
Fold: 2
Root MSE: 4.1003947257995605
Fold: 3
Root MSE: 4.252614974975586
Fold: 4
Root MSE: 4.507584571838379
Fold: 5
Root MSE: 3.7096540927886963
Mean Performance: 4.2535524
Processed 2.000 out of 150.000 elements
Fold: 1
Root MSE: 15.406168937683105
Fold: 2
Root MSE: 15.32197093963623
Fold: 3
Root MSE: 13.83474063873291
Fold: 4
Root MSE: 8.856071472167969
Fold: 5
Root MSE: 19.63810157775879
Mean Performance: 14.61141
Processed 3.000 out of 150.000 elements
Fold: 1
Root MSE: 5.1722211837768555
Fold: 2
Root MSE: 4.801832675933838
Fold: 3
Root MSE: 5.718461990356445
Fold: 4
Root MSE: 5.352035045623779
Fold: 5
Root MSE: 5.3212127685546875
Mean Performance: 5.273153
Processed 4.000 out of 150.000 elements
Fold: 1